In [1]:
import pandas as pd
from sklearn_crfsuite import CRF

In [119]:
# # 读取语料
# def read_corpus(sents_path, tags_path):
#     corpus = []
#     with open(sents_path, 'r', encoding='utf-8') as f1, open(tags_path, 'r', encoding='utf-8') as f2:
#         for line_sent, line_tag in zip(f1.readlines(), f2.readlines()):
#             line_sent = line_sent.strip().split()
#             line_tag = line_tag.strip().split()
#             assert len(line_sent) == len(line_tag)
#             corpus.append((line_sent, line_tag))
#     return corpus


# # 读取标签信息
# def read_tags(file_path):
#     tags = []
#     with open(file_path, 'r', encoding='utf-8') as f:
#         for line in f.readlines():
#             line = line.strip()
#             tags.append(line)
#     return tags

# train_sent_path = 'dataset/MSRA/train/sentences.txt'
# train_tag_path = 'dataset/MSRA/train/tags.txt'

# test_sent_path = 'dataset/MSRA/test/sentences.txt'
# test_tag_path = 'dataset/MSRA/test/tags.txt'

# train_corpos = read_corpus(train_sent_path, train_tag_path)
# test_corpos = read_corpus(test_sent_path, test_tag_path)

In [120]:
# with open('dataset/train.csv', 'w', encoding='utf-8') as f:
#     f.write('Sentence_Index,Token,Tag\n')
#     id = 0
#     for c in train_corpos:
#         for i in range(len(c[0])):
#             f.write(str(id)+','+c[0][i]+','+c[1][i]+'\n')
#         id += 1

# with open('dataset/test.csv', 'w', encoding='utf-8') as f:
#     f.write('Sentence_Index,Token,Tag\n')
#     id = 0
#     for c in test_corpos:
#         for i in range(len(c[0])):
#             f.write(str(id)+','+c[0][i]+','+c[1][i]+'\n')
#         id += 1

In [2]:
train_df = pd.read_csv('dataset/train.csv')
train_df['Token'].fillna('NA', inplace=True)
test_df = pd.read_csv('dataset/test.csv')
test_df['Token'].fillna('NA', inplace=True)

In [3]:
train_df.head()

,Sentence_Index,Token,Tag
0,0,如,O
1,0,何,O
2,0,解,O
3,0,决,O
4,0,足,O


In [123]:
test_df.head()

,Sentence_Index,Token,Tag
0,0,中,B-ORG
1,0,共,I-ORG
2,0,中,I-ORG
3,0,央,I-ORG
4,0,致,O


In [4]:
def extract_sentence_features(df):
    '''
    每个标记的特征是基于其自身以及上一个和下一个标记的上下文信息。
    '''
    sentence_length = len(df)

    for i in range(sentence_length):
        token = df.iloc[i]['Token']

        features = {
            'bias': 1.0,    # 偏置项，用于模型的训练。
            'w':token,
            # 'token.isdigit()': token.isdigit()
        }

        # if i > 0:：如果不是句子的第一个标记，执行以下代码块：
        # 获取前一个标记的文本内容，并添加以下特征：
        if i > 0:
            previous_token = df.iloc[i-1]['Token']
            features.update({
                'w-1':previous_token,
                'w-1:w':previous_token+token,
                # 'previous_token.isdigit()': previous_token.isdigit()
            })
        # 如果是句子的第一个标记，添加特殊特征 'BOS'（Beginning of Sentence）。
        else:
            features['BOS'] = True

        # 如果不是句子的最后一个标记，执行以下代码块：
        # 获取后一个标记的文本内容，并添加以下特征：
        if i < sentence_length - 1:
            posterior_token = df.iloc[i+1]['Token']
            features.update({
                'w+1':posterior_token,
                'w+1:w':posterior_token+token,
                # 'posterior_token.isdigit()': posterior_token.isdigit()
            })
        # 如果是句子的最后一个标记，添加特殊特征 'EOS'（End of Sentence）。
        else:
            features['EOS'] = True

        yield features

In [5]:
def prepare(df, include_y=False):
    X, y = [], []
    for _, group_df in df.groupby(['Sentence_Index']):
        X.append(list(extract_sentence_features(group_df)))
        if include_y:
            y.append(group_df['Tag'])
    if include_y:
        return X, y
    return X

In [6]:
X_train, y_train = prepare(train_df, include_y=True)
X_test, y_test = prepare(test_df, include_y=True)

In [7]:
# len(X_train)
len(X_test)

3442

In [16]:
crf = CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    delta=0.001,
    all_possible_transitions=True,
    verbose=True,
)

In [17]:
# crf.fit(X_train, y_train)
try:
    crf.fit(X_train, y_train)
except AttributeError:
    pass


loading training data to CRFsuite: 100%|██████████| 42000/42000 [00:09<00:00, 4333.88it/s]



Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 665358
Seconds required: 2.715

L-BFGS optimization
c1: 0.300000
c2: 0.100000
num_memories: 6
max_iterations: 100
epsilon: 0.000010
stop: 10
delta: 0.001000
linesearch: MoreThuente
linesearch.max_iterations: 20

Iter 1   time=1.47  loss=1819823.93 active=657735 feature_norm=1.00
Iter 2   time=0.82  loss=1541376.29 active=650082 feature_norm=4.21
Iter 3   time=0.76  loss=1328406.03 active=651920 feature_norm=3.67
Iter 4   time=3.98  loss=1057278.84 active=310152 feature_norm=1.99
Iter 5   time=0.74  loss=1046701.57 active=665298 feature_norm=3.30
Iter 6   time=0.71  loss=878787.11 active=665281 feature_norm=3.18
Iter 7   time=2.81  loss=703666.26 active=582539 feature_norm=3.34
Iter 8   time=1.39  loss=699062.84 active=551127 feature_norm=3.90
Iter 9   time=0.73  loss=645768.00 active=621027 feature_

In [33]:
from collections import Counter

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top likely transitions:")
print_transitions(Counter(crf.transition_features_).most_common(20))

print("\nTop unlikely transitions:")
print_transitions(Counter(crf.transition_features_).most_common()[-20:])


Top likely transitions:
I-ORG  -> I-ORG   6.266624
B-PER  -> I-PER   3.919506
I-PER  -> I-PER   3.626846
O      -> O       3.560464
B-ORG  -> I-ORG   3.428959
I-LOC  -> I-LOC   2.929716
B-LOC  -> I-LOC   2.910829
O      -> B-ORG   1.491918
O      -> B-PER   0.588304
O      -> B-LOC   0.410751
I-LOC  -> O       -0.077332
I-LOC  -> B-LOC   -0.078167
I-PER  -> O       -0.092089
I-ORG  -> O       -0.248649
I-PER  -> B-PER   -0.745519
B-LOC  -> B-LOC   -0.927507
I-LOC  -> B-ORG   -1.144386
I-PER  -> B-ORG   -1.279218
I-ORG  -> B-ORG   -1.338060
B-LOC  -> O       -1.516935

Top unlikely transitions:
B-ORG  -> B-PER   -4.680955
B-ORG  -> O       -4.746733
B-PER  -> B-LOC   -4.811949
B-PER  -> I-ORG   -4.820961
I-ORG  -> I-PER   -4.978876
I-PER  -> I-ORG   -5.169316
B-LOC  -> B-PER   -5.291380
B-ORG  -> I-PER   -5.564023
B-LOC  -> I-PER   -5.709079
B-ORG  -> B-LOC   -5.917233
I-LOC  -> I-PER   -5.987680
I-PER  -> I-LOC   -6.282041
I-ORG  -> I-LOC   -6.506312
B-PER  -> I-LOC   -6.591651
B-ORG  

In [34]:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-8s %s" % (weight, label, attr))    

print("Top positive:")
print_state_features(Counter(crf.state_features_).most_common(30))

print("\nTop negative:")
print_state_features(Counter(crf.state_features_).most_common()[-30:])


Top positive:
9.761739 O        w:、
8.312941 O        w:，
8.256189 B-PER    w-1:w:马列
8.246758 B-PER    w:袁
7.820297 B-PER    w:刘
7.388132 B-PER    w:赵
7.378631 I-LOC    w-1:w:埃及
7.344671 B-PER    w:薛
7.089330 B-ORG    w+1:w:指道
7.073043 B-ORG    w+1:w:大联
7.023772 B-ORG    w+1:w:合共
7.003327 O        w:某
6.948690 B-LOC    w:淮
6.863508 O        w:的
6.774855 B-PER    w-1:w:老于
6.705571 O        w:等
6.490065 B-PER    w:邓
6.397761 O        w:在
6.244106 B-PER    w:郭
6.239973 O        w:与
6.113476 I-ORG    w-1:w:一汽
6.034832 O        w:是
5.995766 B-PER    w:靳
5.972042 B-LOC    w+1:w:药中
5.928725 B-ORG    w-1:w:入盟
5.926247 O        EOS
5.890072 B-PER    w:李
5.780547 O        w:对
5.756613 O        w+1:w:区市
5.724364 B-LOC    w+1:w:方中

Top negative:
-4.403203 O        w+1:w:会国
-4.411551 O        w+1:w:外中
-4.434201 O        w:厄
-4.438736 O        w:瑞
-4.484127 O        w:德
-4.491924 O        w:沪
-4.533850 O        w:萨
-4.575599 O        w:曼
-4.611519 O        w:曹
-4.723146 I-ORG    w-1:队
-4.744141 O   

In [18]:
y_pred = crf.predict(X_test)

In [19]:
from seqeval.metrics import classification_report
list_y_test = [s.tolist() for s in y_test]

report = classification_report(list_y_test, y_pred)
print(report)

              precision    recall  f1-score   support

         LOC       0.93      0.85      0.89      2886
         ORG       0.80      0.77      0.79      1331
         PER       0.85      0.67      0.75      1973

   micro avg       0.88      0.77      0.82      6190
   macro avg       0.86      0.76      0.81      6190
weighted avg       0.88      0.77      0.82      6190



In [20]:
import pickle
s = pickle.dumps(crf)
with open('crf.model','wb+') as f:
	f.write(s)

In [21]:
import pickle
f = open('crf.model','rb') #注意此处model是rb
s = f.read()
model = pickle.loads(s)

In [32]:
import pandas as pd


def pred_sentence(data):
    df = pd.DataFrame(data) 
    # 调用 extract_sentence_features 函数生成特征 
    sentence_features = list(extract_sentence_features(df))
    test = []
    test.append(sentence_features)
    pred = crf.predict(test)
    return pred

def pred_result(tags):
    output = ''
    idx = 0
    for idx in range(len(tags)):
        if tags[idx] != 'O':
            output += Tokens[idx]
            if (idx+1)<len(tags) and tags[idx+1]=='O':
                print('\t'+Tags[idx][2:] + ': ' + output)
        else:
            output = ''


test_df = pd.read_csv('dataset/test.csv')
data = test_df[test_df['Sentence_Index']==90]
pred = pred_sentence(data)

str = ''
for t in list(data['Token']):
    str += t
print(str)
Tokens = list(data['Token'])
Tags = list(data['Tag'])
preds = list(pred[0])

print('test data: ')
pred_result(Tags)
print('\npred result: ')
pred_result(preds)
    

本报洛杉矶１１月２日电记者陈特安、何洪泽报道：今天中午，洛杉矶市民议政论坛、亚洲协会南加中心、美中关系全国委员会、美中友协美西分会等四个友好团体在比佛利山的希尔顿饭店举行盛大午宴，热烈欢迎正在洛杉矶访问的中国国家主席江泽民及其一行。
test data: 
	LOC: 洛杉矶
	PER: 陈特安
	PER: 何洪泽
	ORG: 洛杉矶市民议政论坛
	ORG: 亚洲协会南加中心
	ORG: 美中关系全国委员会
	ORG: 美中友协美西分会
	LOC: 比佛利山
	ORG: 希尔顿饭店
	LOC: 洛杉矶
	LOC: 中国
	PER: 江泽民

pred result: 
	LOC: 洛杉矶
	PER: 陈特安
	PER: 何洪泽
	ORG: 洛杉矶市民议政论坛
	ORG: 亚洲协会南加中心
	ORG: 美中
	ORG: 全国委员会
	ORG: 美中友协美西分会
	LOC: 比佛利山
	ORG: 希尔顿饭店
	LOC: 洛杉矶
	LOC: 中国
	PER: 江泽民
